In [1]:
#essentails
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

#Models
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor
from sklearn.svm import SVR, SVC


#metrics and model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

#Plotting and Feature Selection
import eli5
# import pdpbox
# from pdpbox import pdp, info_plots
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import RFECV

In [2]:
data = pd.read_csv("epl_matches_train.csv")
data.head()

,season,stage,date,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,...,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team,home_team_goal,away_team_goal
0,2008/2009,1,2008-08-17 00:00:00,49337,10260,10261,1,2,4,6,...,0,0,24,9,6,6,55.0,45.0,1,1
1,2008/2009,1,2008-08-16 00:00:00,38136,9825,8659,1,2,4,6,...,0,0,21,7,7,5,66.0,34.0,1,0
2,2008/2009,1,2008-08-16 00:00:00,43276,8472,8650,1,2,4,6,...,0,0,15,19,1,8,46.0,54.0,0,1
3,2008/2009,1,2008-08-16 00:00:00,40671,8654,8528,1,2,4,6,...,0,0,15,27,6,10,52.0,48.0,2,1
4,2008/2009,1,2008-08-17 00:00:00,34633,10252,8456,1,2,4,6,...,0,0,16,16,7,8,52.0,48.0,4,2


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 90 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   season                     2660 non-null   object 
 1   stage                      2660 non-null   int64  
 2   date                       2660 non-null   object 
 3   match_id                   2660 non-null   int64  
 4   home_team_id               2660 non-null   int64  
 5   away_team_id               2660 non-null   int64  
 6   home_player_X1             2660 non-null   int64  
 7   home_player_X2             2660 non-null   int64  
 8   home_player_X3             2660 non-null   int64  
 9   home_player_X4             2660 non-null   int64  
 10  home_player_X5             2660 non-null   int64  
 11  home_player_X6             2660 non-null   int64  
 12  home_player_X7             2660 non-null   int64  
 13  home_player_X8             2660 non-null   int64

In [4]:
data.columns

Index(['season', 'stage', 'date', 'match_id', 'home_team_id', 'away_team_id',
       'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10', 'home_player_X11',
       'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
       'home_player_2

In [5]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

In [6]:
data = data.drop(columns = ['date'] , axis =1)
data = data.drop(columns = ['season'] , axis =1)

In [7]:
def func(x):
    if x<0:
        return 0
    elif x==0:
        return 1
    else:
        return 2

In [8]:
data['Value'] = data['home_team_goal']-data['away_team_goal']
data['Label'] = data['Value'].apply(lambda x: func(x))
data.head()

,stage,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,...,corner_home_team,corner_away_team,possession_home_team,possession_away_team,home_team_goal,away_team_goal,month,year,Value,Label
0,1,49337,10260,10261,1,2,4,6,8,2,...,6,6,55.0,45.0,1,1,8,2008,0,1
1,1,38136,9825,8659,1,2,4,6,8,2,...,7,5,66.0,34.0,1,0,8,2008,1,2
2,1,43276,8472,8650,1,2,4,6,8,2,...,1,8,46.0,54.0,0,1,8,2008,-1,0
3,1,40671,8654,8528,1,2,4,6,8,2,...,6,10,52.0,48.0,2,1,8,2008,1,2
4,1,34633,10252,8456,1,2,4,6,8,2,...,7,8,52.0,48.0,4,2,8,2008,2,2


In [9]:
data = data.dropna()

## MODEL BUILDING

In [10]:
train = data[data['year']!=2014]
y_train = train["Label"]
print(train.shape,y_train.shape)

(2206, 92) (2206,)


In [11]:
test = data[data['year']== 2014]
y_test = test["Label"]
print(test.shape,y_test.shape)

(373, 92) (373,)


In [12]:
X_train = train[['year','match_id',  'home_team_id', 'away_team_id',
        'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10', 'home_player_X11',
       'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11' ]]

In [13]:
X_test = test[['year','match_id',  'home_team_id', 'away_team_id',
        'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10', 'home_player_X11',
       'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11'
       ]]

In [14]:
X_train.shape

(2206, 70)

In [15]:
X_test.shape

(373, 70)

### Model 

In [16]:
# xgb = XGBClassifier(random_state=0)
# xgb.fit(X_train, y_train)
# preds = xgb.predict(X_test)
# print("Accuracy: ", accuracy_score(y_test, preds))

In [17]:
# parm = PermutationImportance(xgb).fit(X_test, y_test)
# eli5.show_weights(parm, feature_names=X_test.columns.tolist())

In [18]:
# plt.figure(figsize=(10,10))
# sns.barplot(x=(xgb.feature_importances_)/sum(xgb.feature_importances_), y=X_train.columns)

In [19]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, preds))

Accuracy:  0.5013404825737265


### Hypertuning Random_Forest Classifier (Not worked: Getting only 2 classifications)

In [20]:
# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# # Number of features to consider at every split
# max_features = ['sqrt']
# # Maximum number of levels in tree
# max_depth = [2,4]
# # Minimum number of samples required to split a node
# min_samples_split = [3, 5]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [2,3]
# # Method of selecting samples for training each tree


In [21]:
# # Create the param grid
# param_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf}

In [22]:
# rf_Model = RandomForestClassifier()

In [23]:
# from sklearn.model_selection import GridSearchCV
# rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, n_jobs = 4)

In [24]:
# rf_Grid.fit(X_train, y_train)

In [25]:
# rf_Grid.best_params_

In [26]:
# rf_Grid.score(X_train,y_train)

In [27]:
# preds = rf_Grid.predict(X_test)
# preds


In [28]:
# ada = AdaBoostClassifier()
# ada.fit(X_train, y_train)
# preds = ada.predict(X_test)
# print("Accuracy: ", accuracy_score(y_test, preds))

In [29]:
# lgb = LGBMClassifier(random_state=0)
# lgb.fit(X_train, y_train)
# preds = lgb.predict(X_test)
# print("Accuracy: ", accuracy_score(y_test, preds))

### IMPORTING TEST CSV FILE & APPLYING MODEL ON IT

In [30]:
test=pd.read_csv('epl_matches_test.csv')
test.shape

(380, 72)

In [31]:
test['date'] = pd.to_datetime(test['date'], errors='coerce')
data['month'] = test['date'].dt.month
test['year'] = test['date'].dt.year

In [32]:
test_data= test[['year','match_id',  'home_team_id', 'away_team_id',
        'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10', 'home_player_X11',
       'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11' ]]

In [33]:
test_data=test_data.fillna(0)
test_data.shape

(380, 70)

In [34]:
test_preds = rf.predict(test_data)

In [35]:
test_preds

array([2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 0, 0,
       1, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0,
       0, 1, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2,
       0, 1, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0,
       0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2,
       2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 0, 2, 0, 2,
       2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0,

In [36]:
test_preds = pd.DataFrame(test_preds, columns = ['result'])

In [37]:
test_preds.replace(to_replace = {0:'lose',1:'draw',2:'win'},inplace=True )

In [38]:
prediction_submission=pd.read_csv('prediction_submission.csv')

In [39]:
prediction_submission['match_id']= test_data['match_id']

In [40]:
prediction_submission['result'] = test_preds['result']

In [41]:
prediction_submission.head(10)

,match_id,result
0,48994,win
1,54777,win
2,31434,win
3,56248,win
4,44311,win
5,49672,win
6,48506,lose
7,45922,lose
8,45509,win
9,48741,lose


### Saving FIle to destination folder

In [42]:
prediction_submission.to_csv('prediction_submission.csv', index= False)